<a href="https://colab.research.google.com/github/yongsuk-lee-korea/insurance-house/blob/main/Llama3_%ED%8C%8C%EC%9D%B8%ED%8A%9C%EB%8B%9D%EC%9A%A9_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%83%9D%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **환경설정 (본인 구글드라이브 내 dataset 폴더**

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [15]:
dataPath = '/content/gdrive/MyDrive/Llama3 dataset/'

In [3]:
!pip install jsonlines

In [4]:
import huggingface_hub
huggingface_hub.login()

In [5]:
import pandas as pd
import json

In [6]:
import jsonlines

In [9]:
!pip install datasets==2.19.0

In [8]:
!pip install datasets

In [10]:
from datasets import Dataset

In [11]:
!pip install gspread gspread_dataframe


In [12]:
#데이터 경로 및 설정 불러오기
datasetName = "insurance-kics.csv"
jsonFileName = "insurance-kics.jsonl"

In [16]:
def csv_to_json(csv_file_path, json_file_path):
    try:
        df = pd.read_csv(csv_file_path, encoding='utf-8')  # 인코딩을 명시적으로 지정
    except UnicodeDecodeError:
        df = pd.read_csv(csv_file_path, encoding='cp949')  # 다른 인코딩을 시도
    with open(json_file_path, "w", encoding='utf-8') as json_file:
        for index, row in df.iterrows():
            data = {'Inputs': row['inputs'], 'response': row['response']}
            json.dump(data, json_file, ensure_ascii=False)
            json_file.write("\n")

csv_file_path = dataPath + datasetName
json_file_path = dataPath + jsonFileName

csv_to_json(csv_file_path, json_file_path)


In [17]:
!pip install gspread gspread_dataframe


**파인튜닝용 포맷변환(json)**

In [18]:
import jsonlines
from datasets import Dataset

# JSONLines 파일을 읽어서 데이터셋 생성
indataset = []
with jsonlines.open(json_file_path) as f:
    for line in f.iter():
        indataset.append(f"<s>[INST] {line['Inputs']} [/INST] {line['response']} </s>")

print('데이터셋 확인')
print(indataset[:5])

# 데이터셋을 사전으로 변환하여 Hugging Face Dataset으로 저장
indataset = Dataset.from_dict({'text': indataset})
indataset.save_to_disk(dataPath)

print('데이터셋 info 확인')
print(indataset)


데이터셋 확인
['<s>[INST] 보험회사란 무엇이지? [/INST] 보험업법에 의한 국내 보험회사와 해외 소재지국의 보험업 관련 법규에 의한 해외 보험회사를 말한다. </s>', '<s>[INST] 이용석 이사 아들은 몇 명이지? [/INST] 두명 </s>', '<s>[INST] 이용석 이사의 첫째 아들은 몇 년 생이지? [/INST] 2020년 3월 17일 입니다. </s>', '<s>[INST] 이용석 이사의 첫째 아들의 이름은 무엇이지? [/INST] 이윤재 </s>', '<s>[INST] 이용석 이사의 둘째 아들은 몇 년 생이지? [/INST] 2022년 12월 14일 입니다. </s>']


Saving the dataset (0/1 shards):   0%|          | 0/23 [00:00<?, ? examples/s]

데이터셋 info 확인
Dataset({
    features: ['text'],
    num_rows: 23
})


In [19]:
indataset

Dataset({
    features: ['text'],
    num_rows: 23
})

In [21]:
# 허깅페이스에 업로드
indataset.push_to_hub("ysleekr/insurance-kics-guidance")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/ysleekr/insurance-kics-guidance/commit/091179775d56d19d4fe95e176424f422950192af', commit_message='Upload dataset', commit_description='', oid='091179775d56d19d4fe95e176424f422950192af', pr_url=None, pr_revision=None, pr_num=None)